In [1]:
%matplotlib widget
import numpy as np
import os
import h5py
import subprocess

print(os.getpid())

%cd ../                  
import pylib.mix as mix

27147
/media/work/docs/codes/QuCF/scripts-py


Compute the QSVT angles (phase factors) by using the [Dong algorithm](https://journals.aps.org/pra/abstract/10.1103/PhysRevA.103.042419).

In [2]:
# -----------------------------------------------------
# --- Helper functions ---
# -----------------------------------------------------
def choose_function(sel_function):
    if mix.compare_two_strings(sel_function, "inversion"):
        name_par, short_par_id = "condition_number", "k"

    return sel_function, name_par, short_par_id
# -----------------------------------------------------
def print_input_config():
    print("function selector: {:s}".format(sel_function_))
    print("parameter [{:s}] value: {:0.3e}".format(name_par_, parameter_value_))
    print("work directory: {:s}".format(work_dir_))
    print("name of the output file with Cheb. coefs.: {:s}".format(filename_poly_))
    return
# -----------------------------------------------------
def compute_Cheb_coefs():
    exe_cmd = "../QuCF/build_polynomial/approx_polyn"

    line_cmd = "{:s} -sel_function {:s} -param {:0.12e} -Nd {:d} -work_path {:s} -filename_out {:s}".format(
        exe_cmd,
        sel_function_, 
        parameter_value_, 
        N_Chebyschev_coefs_,
        work_dir_,
        filename_poly_
    )

    res_proc = subprocess.run(
        line_cmd, 
        shell = True, 
        executable="/bin/bash"
    )
    return
# -----------------------------------------------------
def compute_QSVT_angles(filename_angles):
    # --- Create .ca file (input file for computing the angles) ---
    text_ca = ""
    text_ca += "filename_coefs {:s}".format(work_dir_ + filename_poly_) + "\n"
    text_ca += "output_name    {:s}".format(work_dir_ + filename_angles) +"\n"
    text_ca += "stopping_criterion 1e-12" + "\n"

    filename_ca = "calc_{:s}".format(sel_function_)
    with open(work_dir_ + filename_ca + ".ca", "w") as file:
        file.write(text_ca)

    # --- Computation ---
    exe_cmd = "../QuCF/build_angles/compute_angles"

    line_cmd = "{:s} {:s} ./".format(
        exe_cmd,
        work_dir_ + filename_ca, 
    )

    res_proc = subprocess.run(
        line_cmd, 
        shell = True, 
        executable="/bin/bash"
    )
    return

In [3]:
# ----------------------------------------------------------
# --- Input parameters ---
# ----------------------------------------------------------
root_dir_ = "./tools/QSVT-angles/"
sel_method_ = "cpp_pol"  


sel_function_, name_par_, short_par_id_ = choose_function("inversion")

parameter_value_, N_Chebyschev_coefs_ = 20., 40
# parameter_value_, N_Chebyschev_coefs_ = 100., 2085

work_dir_ = root_dir_ + "{:s}/test".format(sel_function_)

filename_poly_ = "poly_{:s}_{:0.0f}_Nc_{:d}".format(short_par_id_, parameter_value_, N_Chebyschev_coefs_)

# --- Print input data ---
print_input_config()
work_dir_ += "/"

function selector: inversion
parameter [condition_number] value: 1.000e+02
work directory: ./tools/QSVT-angles/inversion/test
name of the output file with Cheb. coefs.: poly_k_100_Nc_2085


In [4]:
# ----------------------------------------------------------
# --- Compute Chebyschev coefficients ---
# ----------------------------------------------------------
compute_Cheb_coefs()

--- Fourier approach ---
Function to approximate: inversion
Its parity: 	1
Parameter: 	100
Rescaling parameter: 	0.125
Nd: 	2085
N of avail. GPU devices: 1
work directory: /media/work/docs/codes/QuCF/scripts-py/./tools/QSVT-angles/inversion/test/
output filename: poly_k_100_Nc_2085

Avail. GPU mem for coefs. (MB): 7164

---
Required mem. for coefs. (MB): 0.0158997

	 *** Iteration: 0 ***
	N-coefs per device: 1042
	N-blocks: 2
	N-threads: 1024

Construction of the polynomial...
Resulting approximation error: 7.869e-04
Data are saved to the file: ./tools/QSVT-angles/inversion/test//poly_k_100_Nc_2085.hdf5


In [5]:
# ----------------------------------------------------------
# --- Compute QSVT angles using the computed QSVT angles ---
# ----------------------------------------------------------
with h5py.File(work_dir_ + filename_poly_ + ".hdf5", "r") as f:
    gr = f["basic"]
    eps_res_ = np.abs(gr["eps"][()])
print()
print("abs. error in Cheb. approx.: {:0.3e}".format(eps_res_))
print("log10(abs. error): {:0.3e}".format(np.log10(eps_res_)))

# --- The file name of the .hdf5 file where the resulting QSVT angles will be saved ---
eps_res_log = int(np.round(np.log10(eps_res_)))
filename_angles = "angles_{:s}{:0.0f}_eps{:d}".format(short_par_id_, parameter_value_, abs(eps_res_log))

# --- Computation ---
compute_QSVT_angles(filename_angles)


abs. error in Cheb. approx.: 7.869e-04
log10(abs. error): -3.104e+00


********************************************************
********************************************************
Project name: ./tools/QSVT-angles/inversion/test/calc_inversion

Reading input parameters from the ./tools/QSVT-angles/inversion/test/calc_inversion.ca file in the directory:
/media/work/docs/codes/QuCF/scripts-py

	Reading coefficients from the file: ././tools/QSVT-angles/inversion/test/poly_k_100_Nc_2085
	The coefficients were computed on: 01-03-2025 18:17:50
	Done.
herefunction type: 	inversion
parity: 	1
parameter: 	100
absolute error of polynomial approximation: 7.869e-04
coefficient of normalization: 	0.125
number of coefficients: 	1042
number of L-BFGS pairs: 	200
stopping criterion square: 1.000e-24
Output file name: ./tools/QSVT-angles/inversion/test/angles_k100_eps3.hdf
Done.
Creating output ./tools/QSVT-angles/inversion/test/calc_inversion.hdf5 file in the directory:
/media/work/docs/codes/Qu